In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

# Ekstraksi data dari halaman web menjadi DataFrame
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    
    # Ambil seluruh elemen <tbody> dan pilih tbody pertama
    tbody_elements = data.find_all('tbody')
    if len(tbody_elements) > 0:
        rows = tbody_elements[0].find_all('tr')
        for row in rows:
            col = row.find_all('td')
            if len(col) > 2 and len(col) != 0:
                # Ekstrak data Rank, Bank Name, dan Market Capitalization (MC) dalam USD
                rank = col[0].get_text(strip=True)
                bank_name = col[1].get_text(strip=True)
                mc_value = col[2].get_text(strip=True)
                
                # Menambahkan data yang valid ke dalam DataFrame
                if bank_name and mc_value and '—' not in mc_value:
                    data_dict = {"Rank": rank, "Name": bank_name, "MC_USD_Billion": mc_value}
                    df = pd.concat([df, pd.DataFrame(data_dict, index=[0])], ignore_index=True)
    else:
        print("Tidak ada elemen <tbody> ditemukan.")
    
    return df

# Transformasi data
def transform(df):
    # Mengubah nilai MC_USD_Billion menjadi angka dan mengonversinya ke dalam format yang sesuai
    df["MC_USD_Billion"] = df["MC_USD_Billion"].apply(lambda x: float(x.replace(',', '').replace('$', '').strip()))
    
    # Menambahkan kolom MC_GBP_Billion, MC_EUR_Billion, dan MC_INR_Billion berdasarkan nilai tukar
    exchange_rates = {'GBP': 0.75, 'EUR': 0.92, 'INR': 82.5}  # Nilai tukar terhadap USD
    df['MC_GBP_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rates['GBP'], 2)
    df['MC_EUR_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rates['EUR'], 2)
    df['MC_INR_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rates['INR'], 2)
    
    return df

# Simpan DataFrame ke file CSV
def load_to_csv(df, csv_path):
    df.to_csv(csv_path, index=False)

# Simpan DataFrame ke database SQLite
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

# Fungsi untuk log progres
def log_progress(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open("./etl_project_log.txt", "a") as f:
        f.write(f"{timestamp} : {message}\n")

# Task 6: Fungsi untuk menjalankan query dan mencetak hasil
def run_queries(query_statement, sql_connection):
    # Menjalankan query
    query_output = pd.read_sql(query_statement, sql_connection)
    
    # Mencetak pernyataan query dan hasilnya
    print(f"Query: {query_statement}")
    print(f"Output:\n{query_output}\n")
    
    # Log progres
    log_progress(f"Executed query: {query_statement}")

# Konfigurasi awal dan eksekusi ETL
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'  # URL yang sesuai
table_attribs = ["Rank", "Name", "MC_USD_Billion"]
csv_path = './Largest_Banks.csv'
db_name = 'Banks_Market_Capitalization.db'
table_name = 'Largest_Banks'

# Mulai ETL
log_progress('ETL process started')
df = extract(url, table_attribs)
log_progress('Data extraction complete')
df = transform(df)
log_progress('Data transformation complete')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV')

# Simpan ke database
sql_connection = sqlite3.connect(db_name)
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database')

# Menjalankan query untuk Task 6
run_queries("SELECT * FROM Largest_Banks", sql_connection)
run_queries("SELECT AVG(MC_GBP_Billion) FROM Largest_Banks", sql_connection)
run_queries("SELECT Name FROM Largest_Banks LIMIT 5", sql_connection)

# Akhir proses ETL
log_progress('Process complete')
sql_connection.close()


Query: SELECT * FROM Largest_Banks
Output:
  Rank                                     Name  MC_USD_Billion  \
0    1                           JPMorgan Chase          432.92   
1    2                          Bank of America          231.52   
2    3  Industrial and Commercial Bank of China          194.56   
3    4               Agricultural Bank of China          160.68   
4    5                                HDFC Bank          157.91   
5    6                              Wells Fargo          155.87   
6    7                        HSBC Holdings PLC          148.90   
7    8                           Morgan Stanley          140.83   
8    9                  China Construction Bank          139.82   
9   10                            Bank of China          136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          324.69          398.29        35715.90  
1          173.64          213.00        19100.40  
2          145.92          179.00        16051.20  
3          